Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Configuration

## Introduction
This tutorial shows how to connect azure notebooks with Azure Machine Learning Service workspace and to create Azure Machine Learning Compute (AmlCompute).

Before getting started, we need to make a directory named **`aml_config`**. In this folder, we will save a **`config.json`** file which can be downloaded in the azure portal.

In [ ]:
import os

os.makedirs('aml_config')

## Configure your Azure ML workspace

### Access your workspace

The following cell uses the Azure ML SDK to load the workspace specified by your parameters. If this cell succeeds, your notebook library will be configured to access the workspace from all notebooks using the `Workspace.from_config()` method

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

### Create compute resources for your training experiments

Many of the sample notebooks use Azure ML managed compute (AmlCompute) to train models using a dynamically scalable pool of compute. In this section you will create default compute clusters for use by the other notebooks and any other operations you choose.

To create a cluster, you need to specify a compute configuration that specifies the type of machine to be used and the scalability behaviors.  Then you choose a name for the cluster that is unique within the workspace that can be used to address the cluster later.

The cluster parameters are:
* vm_size - this describes the virtual machine type and size used in the cluster.  All machines in the cluster are the same type.  You can get the list of vm sizes available in your region by using the CLI command

```shell
az vm list-skus -o tsv
```
* min_nodes - this sets the minimum size of the cluster.  If you set the minimum to 0 the cluster will shut down all nodes while not in use.  Setting this number to a value higher than 0 will allow for faster start-up times, but you will also be billed when the cluster is not in use.
* max_nodes - this sets the maximum size of the cluster.  Setting this to a larger number allows for more concurrency and a greater distributed processing of scale-out jobs.


To create a **CPU** cluster now, run the cell below. The autoscale settings mean that the cluster will scale down to 0 nodes when inactive and up to 4 nodes when busy.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

c_name = 'cpucluster'

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=c_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=0,
                                                           max_nodes=4)  ## Subject to be changed

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, c_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

To create a **GPU** cluster, run the cell below. Note that your subscription must have sufficient quota for GPU VMs or the command will fail. To increase quota, see [these instructions](https://docs.microsoft.com/en-us/azure/azure-supportability/resource-manager-core-quotas-request). 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

c_name = 'gpucluster'

try:
    gpu_cluster = ComputeTarget(workspace=ws, name=c_name)
    print("Found existing gpucluster")
except ComputeTargetException:
    print("Creating new gpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           min_nodes=0,
                                                           max_nodes=4)  ## Subject to be changed

    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, c_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)